In [1]:
import os
import sys

import pandas as pd

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

import datetime
from utils import get_path, DATETIME_FORMAT

In [2]:
from universal_connection import UniversalConnection, DBType

In [3]:
sql = """
select	a.id app_id,
		a.patient_id patient_id,
		--p_patient.id person_id,
		date(a.finished at time zone 'UTC') cons_date,
		--
		--
		--a."source",
		concat_ws(' ', p_patient.first_name, p_patient.middle_name, p_patient.last_name) patient_fio,
	    p_patient.phone,
	    p_patient.email,
	    a.report_comment 
--
from appointment a
left join patient on a.patient_id = patient.id
left join person p_patient on p_patient.id = patient.person_id 
--
where a.finished at time zone 'UTC' > '2020-03-30'
--
and a.good
  and not (
    a.patient_id in (
        select patient_id
        from patient_categories pcat
        inner join reference rf
           on rf.id=pcat.reference_id
        where rf.code='TEST'
        )
    or lower(a.report_comment)='тест'
    )		
		
		
		
		


"""

In [4]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df_sql = connection.query(sql)
connection.close()

2020-06-10 15:21:48 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-10 15:21:48 creating ssh tunnel to 172.16.100.19 as root...
2020-06-10 15:21:48 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 44463
2020-06-10 15:21:48 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-06-10 15:21:48 @telemed query:
                    select	a.id app_id,
				    		a.patient_id patient_id,
				    		--p_patient.id person_id,
				    		date(a.finished at time zone 'UTC') cons_date,
				    		--
				    		--
				    		--a."source",
				    		concat_ws(' ', p_patient.first_name, p_patient.middle_name, p_patient.last_name) patient_fio,
				    	    p_patient.phone,
				    	    p_patient.email,
				    	    a.report_comment 
				    --
				    from appointment a
				    left join patient on a.patient_id = patient.

In [5]:
df_sql

,app_id,patient_id,cons_date,patient_fio,phone,email,report_comment
0,137211,1568925,2020-04-08,Ярослав Николаевич Лозинский,79091582287,None,Поражение на коже груди .очаг большой около 15...
1,127919,2509970,2020-03-31,Мария Александровна Шамардина,79518565014,shamardina_93@mail.ru,О.ринит? у ребенка 3-х мес жизни\nРекомендован...
2,144566,3484508,2020-04-15,Тахир Мухаммедович Бикбов,79687101977,None,"консультация уролога - очная,\r\nузи простаты,..."
3,136013,3165112,2020-04-07,Владимир Анатольевич Плюснин,79089255484,None,диагностическая гипотеза: инфекция дыхательных...
4,128137,1125192,2020-03-31,АЛЕКСАНДР АЛЕКСАНДРОВИЧ ФОМИНЫХ,79180989263,None,Диагностическая гипотеза: Хронический вазомото...
...,...,...,...,...,...,...,...
109097,254925,2571628,2020-06-08,Оксана Александровна Андреенко,79029487778,andreenko.ok@ya.ru,С учетом представленной информации можно предп...
109098,254195,3498101,2020-06-08,Светлана Валентиновна шинкарец,79672756316,kurskaya84@mail.ru,На коже туловища большое количество невусов ра...
109099,254919,3779550,2020-06-08,Елена Михайлова Васильева,79803753103,,"Жалобы на слабость, головокружение, низкое АД ..."
109100,254121,3780913,2020-06-08,Никита Александрович Незнанов,79168122935,neznanovnikita@gmail.com,Диагностическая гипотеза-Сосудистая головная б...


In [6]:
int_words = []
a = open('interesting_words.txt')
for line in a:
    #print(line)
    int_words.append(line.strip())
a.close()

In [7]:
int_words

['грудной',
 'пневмония',
 'кашель',
 'дыхания',
 'дыхательных',
 'отхаркивающие',
 'одышка',
 'мокроты',
 'дыхательная']

In [9]:
df_sql['interest'] = df_sql['report_comment'].apply(lambda x: 1 if any(word in x for word in int_words) else 0)

In [12]:
df_sql[df_sql.interest == 1].to_excel('full_data.xlsx', index=False)

In [15]:
df_sql[df_sql.interest == 1].groupby(['patient_id', 'patient_fio', 'phone', 'email'])[['interest']].count().reset_index().rename(columns={'interest':'Обращений'})

,patient_id,patient_fio,phone,email,Обращений
0,3112,Максим Викторович Шарапов,79255295917,sharapov1994@yandex.ru,1
1,3423,Екатерина Евгеньевна Курбанова,79119647770,ekaterina.kurbanova@gmail.com,1
2,4594,Азат Мидхатович Сулейманов,79991312000,azazaraza@mail.ru,1
3,4770,Анна Владимировна Вахтерова,79037831717,anna.vakhterova@me.com,1
4,4910,Татьяна Владимировна Быкова,79162079249,tbykova@mail.ru,1
...,...,...,...,...,...
2528,3817670,Анна Андреевна Сергеенко,79533575406,sergeenko5795@gmail.com,1
2529,3848655,Никита Леонидович Шилкин,79773203741,degert@mail.ru,1
2530,3852267,Кристина Андреевна Чехова,79393340878,kristinachehova@gmail.com,2
2531,3879415,Дмитрий Сергеевич Долгов,79263532730,dimadolgov@yandex.ru,1


In [16]:
df_sql[df_sql.interest == 1].groupby(['patient_id', 'patient_fio', 'phone', 'email'])[['interest']].count().reset_index().rename(columns={'interest':'Обращений'}).to_excel('group_data.xlsx', index=False)